## Probar selenium para navegar con incognito y con la libreria time espaciarle los requests

In [1]:
#Librerias
import re
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import time

In [2]:
# Funcion get
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    head = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/59.0.3071.115 Safari/537.36'}
    try:
        with closing(get(url, stream=True, headers=head)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

# Codigo final todo junto

In [ ]:
#Genero una lista con todos los ids que podria llegar a tener la pagina
#ids = list(range(60000))

In [22]:
ids = list(range(10000))

In [54]:
ids = list(range(50000,60000))

In [55]:
%%time
##Loopeo sobre los id de la URL
games = []


for id in ids:
    try:
        url = 'https://howlongtobeat.com/game.php?id='+str(id)
        raw_html = simple_get(url)
        html = BeautifulSoup(raw_html, 'html.parser')

        ## Titulo del juego
        titulo = html.find('div', {'class':"profile_header shadow_text"}).text.split('\n')
        titulo = titulo[1].rstrip()
            #Pasamos a dict
        titulo_dict = {'Titulo' : titulo}


        ## Rating
        det=html.find('div', {'class':'profile_details'})
        details = []
        for i, li in enumerate(det.select('li')):
            dicc = {i: li.text}
            details.append(li.text)
        rating = details[4][0:2]
            #Pasamos a dict
        rating_dict = {'Rating': rating}
        rating_dict


        ## Completion Times
        tiempos = html.find_all('li',class_=re.compile("short time_"))
            #Descompongo cada uno de los tiempos
        times = []
        for n in range(4):
            x = tiempos[n].select('div')
            x = str(x[0])
            x = x.replace('<div>', '')
            x = x.replace('</div>','')
            x = x.rstrip()
            times.append(x)
            #Creo un diccionario con la clave de cada tiempo
        times_dict = {'Main Story'    : times[0],
                      'Main + Extras' : times[1],
                      'Completionist' : times[2],
                      'All Styles'    : times[3]}


        ## Info del juego
        prof = html.find_all('div', {'class':'profile_info'})
            #Creo un diccionario con la info
        info_dict = {'Developer'   : prof[0].contents[2].replace('\n','').rstrip(),
                     'Publishers'  : prof[1].contents[2].replace('\n','').rstrip(),
                     'Playable On' : prof[2].contents[2].replace('\n','').rstrip(),
                     'Genres'      : prof[3].contents[2].replace('\n','').rstrip()}

        ## Junto Todos los diccionarios en uno solo

        games.append({**titulo_dict, **rating_dict, **times_dict, **info_dict})
        time.sleep(2) # Esperar dos segundos entre cada loop
    except:
        pass

Wall time: 3h 59min 37s


In [ ]:
# Primer loop 6hs 56 min
# Segundo Loop 5hs 35 min
# Tercer loop 4hs 56min
# Cuarto loop 9hs 25 min con 20.000
# Quinto loop 3hs 59 min

In [56]:
import pandas as pd
games_df = pd.DataFrame(games)

In [36]:
games

[{'All Styles': '25 Hours',
  'Completionist': '39½ Hours',
  'Developer': 'Mistwalker, AQ Interactive',
  'Genres': ' August 14, 2012',
  'Main + Extras': '25½ Hours',
  'Main Story': '21½ Hours',
  'Playable On': 'Wii, Wii U',
  'Publishers': 'Nintendo',
  'Rating': '82',
  'Titulo': 'The Last Story'},
 {'All Styles': '6 Hours',
  'Completionist': '--',
  'Developer': 'SCi',
  'Genres': ' November 1993',
  'Main + Extras': '--',
  'Main Story': '6 Hours',
  'Playable On': 'Action',
  'Publishers': 'THQ',
  'Rating': '70',
  'Titulo': 'The Lawnmower Man'},
 {'All Styles': '30 Hours',
  'Completionist': '30 Hours',
  'Developer': 'Quest Software',
  'Genres': ' 1988',
  'Main + Extras': '--',
  'Main Story': '--',
  'Playable On': 'Role-Playing',
  'Publishers': 'Epyx',
  'Rating': 'NR',
  'Titulo': 'The Legend of Blacksilver'},
 {'All Styles': '56 Hours',
  'Completionist': '69 Hours',
  'Developer': 'SCEI',
  'Genres': 'Role-playing',
  'Main + Extras': '56½ Hours',
  'Main Story': '

In [57]:
games_df = games_df[['Titulo','Main Story','Main + Extras','Completionist','All Styles','Rating','Developer', 'Publishers', 'Genres']]

In [58]:
games_df

,Titulo,Main Story,Main + Extras,Completionist,All Styles,Rating,Developer,Publishers,Genres
0,The Elmian Warrior,--,--,--,--,NR,The MegaByte,The MegaByte,"July 20, 2017"
1,Sphere Frustration,--,--,--,--,NR,GreatherGames,GreatherGames,"July 14, 2017"
2,Shn!p,--,--,--,--,NR,Martin Knopf,Martin Knopf,"July 14, 2017"
3,Chromatic (2017),--,--,--,--,NR,Marc-Antoine Jacob,MAJ Studio,"July 17, 2017"
4,Too Angry to Space,--,--,--,--,NR,Ak84c,Ak84c,"August 08, 2016"
5,ISEE,--,--,--,--,NR,"Half-Face Production, Nova Dimension",Nova Dimension,"July 31, 2017"
6,The House of Da Vinci,5 Hours,--,7 Hours,5½ Hours,80,Blue Brain Games,"Android, iOS, PC",2.5 Weeks Ago
7,Achievement Hunter: Offensive,--,--,3 Hours,3 Hours,40,Putilin Industries,Putilin Industries,"September 15, 2017"
8,Theseus (2017),2 Hours,--,--,2 Hours,60,Forge Reply,Forge Reply,"Action, Adventure"
9,A Mortician's Tale,56 Mins,1 Hours,1 Hours,1 Hours,68,Laundry Bear Games,Laundry Bear Games,Simulation


In [59]:
games_df.to_csv('C:/Users/rharari/Desktop/Data Science/hltb5.csv', sep='|')

# -------CODIGO ANTERIOR-------------

## Titulo y rating

In [9]:
#Titulo del juego
titulo = html.find('div', {'class':"profile_header shadow_text"}).text.split('\n')
titulo = titulo[1].rstrip()

In [10]:
#Pasamos a dict
titulo_dict = {'Titulo' : titulo}
titulo_dict

{'Titulo': 'The Witcher 3: Wild Hunt'}

In [11]:
#Detalles de rating, backlog, etc
det=html.find('div', {'class':'profile_details'})

In [12]:
det

<div class="profile_details">
<li>2.1K Playing</li>
<li>7K Backlogs</li>
<li>914 Replays</li>
<li>1% Retired
</li>
<li>96% Rating</li>
<li>5.7K Beat</li>
<div class="clear"></div>
</div>

In [60]:
# Rating
details = []
for i, li in enumerate(det.select('li')):
        dicc = {i: li.text}
        details.append(li.text)
rating = details[4][0:2]

In [61]:
#Pasamos a dict
rating_dict = {'Rating': rating}
rating_dict

{'Rating': '94'}

## Completion Times

In [5]:
html

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="css/main_77.css" rel="stylesheet"/>
<link href="css/desktop_77.css" media="screen and (min-width: 769px)" rel="stylesheet"/>
<link href="css/mobile_77.css" media="screen and (max-width: 769px)" rel="stylesheet"/>
<title>How long is Spyro Reignited Trilogy? - HLTB</title> <meta content="How long is Spyro Reignited Trilogy? HowLongToBeat has the answer. Create a backlog, submit your game times and compete with your friends!" name="description"/> <meta content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/>
<link href="img/icons/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="img/icons/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="img/icons/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="img/icons/apple-touch-icon-76x76.png" re

In [12]:
import re

In [9]:
#Tiempos
tiempos = html.find_all('li', {'class':'short time_100','class':'short time_60'})

In [19]:
tiempos = html.find_all('li',class_=re.compile("short time_"))

In [23]:
#Vemos como quedó
tiempos[2]

<li class="short time_60">
<h5>Completionist</h5>
<div>175 Hours </div>
</li>

In [71]:
#Descompongo cada uno de los tiempos
times = []
for i in range(4):
    x = tiempos[i].select('div')
    x = str(x[0])
    x = x.replace('<div>', '')
    x = x.replace('</div>','')
    x = x.rstrip()
    times.append(x)

IndexError: list index out of range

In [25]:
times

['175 Hours']

In [19]:
# Creo un diccionario con la clave de cada tiempo
times_dict = {'Main Story': times[0],
              'Main + Extras' : times[1],
              'Completionist' : times[2],
              'All Styles' : times[3]}
times_dict

{'All Styles': '102 Hours',
 'Completionist': '171 Hours',
 'Main + Extras': '103 Hours',
 'Main Story': '49½ Hours'}

## Informacion adicional del juego

In [20]:
# Info del juego
prof = html.find_all('div', {'class':'profile_info'})

Podemos acceder a cada uno de los elementos loopeando sobre el objeto prof. Por otro lado, mediante el atributo contents y el indice 2, podemos obtener el texto que estamos buscando para cada uno de los atributos del juego

In [44]:
#Creo un diccionario con la info
info_dict = {'Developer'   : prof[0].contents[2].replace('\n','').rstrip(),
             'Publishers'  : prof[1].contents[2].replace('\n','').rstrip(),
             'Playable On' : prof[2].contents[2].replace('\n','').rstrip(),
             'Genres'      : prof[3].contents[2].replace('\n','').rstrip()}

In [46]:
info_dict

{'Developer': 'CD Projekt RED',
 'Genres': 'Third-Person, Action, Adventure, Open World, Role-Playing',
 'Playable On': 'Linux, PC, PlayStation 4, Xbox One',
 'Publishers': 'CD Projekt, Warner Bros. Interactive Entertainment'}

In [47]:
#Para juntar los diccionarios

{**titulo_dict, **rating_dict, **times_dict, **info_dict}

{'All Styles': '102 Hours',
 'Completionist': '171 Hours',
 'Developer': 'CD Projekt RED',
 'Genres': 'Third-Person, Action, Adventure, Open World, Role-Playing',
 'Main + Extras': '103 Hours',
 'Main Story': '49½ Hours',
 'Playable On': 'Linux, PC, PlayStation 4, Xbox One',
 'Publishers': 'CD Projekt, Warner Bros. Interactive Entertainment',
 'Rating': '96',
 'Titulo': 'The Witcher 3: Wild Hunt'}

In [207]:
# Creo un diccionario con la info
keys = ['Developer', 'Publishers', 'Playable On', 'Genres']
info_dict = {}

for i in range(4):
    prof[i].contents[2]
prof[0] ## Developer
prof[1] ## Publishers
prof[2] ## Playable On
prof[3] ##Genres

<div class="profile_info">
<strong>
Genres:
</strong>
Third-Person, Action, Adventure, Open World, Role-Playing </div>

In [3]:
#Raw HTML
raw_html = simple_get('https://howlongtobeat.com/game.php?id=55358')

In [4]:
#Paso el request a un objeto de BeatifulSoup
html = BeautifulSoup(raw_html, 'html.parser')